# Computing scores for recombination

Before you run this notebook, you should have already run:
1. E2_0_Record_and_Segment.ipynb

The generated demonstrations are stored in a directory which will be used here

This notebook does the following:
1. Compute the error matrix between demonstration parts(2 part case)
2. Compute errors of the last part of a demonstration with respect to the goal image

In [ ]:
import os
import copy
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path
import numpy as np

from scipy.spatial.transform import Rotation as R

from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.demo.demo_episode_recorder import record_sim
from flow_control.runner import evaluate_control
from flow_control.servoing.module import ServoingModule
from flow_control.servoing.playback_env_servo import PlaybackEnvServo
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout
import seaborn as sns
from tqdm import tqdm
import getpass

%matplotlib inline

experiment = "multi_shape"
goal = "multi_shape_goal"

def get_data_dir():
    username = getpass.getuser()
    if username == "argusm":
        return "/tmp/flow_experiments3"
    elif username == "nayakab":
        return "../tmp"

data_dir = get_data_dir()

root_dir = os.path.join(data_dir, experiment)
goal_dir = os.path.join(data_dir, goal)

## Load all demonstrations 

In [ ]:
def get_recordings(directory):
    return sorted([os.path.join(directory, rec) for rec in os.listdir(directory) if os.path.isdir(os.path.join(directory, rec))])

recordings = get_recordings(root_dir)
goal_recordings = get_recordings(goal_dir)

In [ ]:
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:]]

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print("wp_name:", playbacks[demo_index][frame_index].get_info()["wp_name"])
    fg_mask = playbacks[demo_index].get_fg_mask()
    if fg_mask is not None:
        print("percent fg:", np.mean(fg_mask)*100)
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

In [ ]:
def filter_demo(pb):
    return pb[-1].data['rew'] > 0 and np.mean(pb.get_fg_mask()) > 0.005

demo_good = [filter_demo(pb) for pb in playbacks]
good_demonstrations = np.where(demo_good)[0]
good_demonstrations = [int(x) for x in good_demonstrations]

In [ ]:
# Load demo segmentation file
demo_seg_file = f'{root_dir}/demo_parts_manual2.json'
fp = open(demo_seg_file)
demo_parts = json.load(fp)
live_indices = [int(key) for key in demo_parts.keys()]

In [ ]:
# Servo Module
# Load Servoing Module
from flow_control.servoing.module import ServoingModule
control_config = dict(mode="pointcloud-abs-rotz", threshold=0.40)
servo_module = ServoingModule(recordings[0], control_config=control_config,
                              start_paused=False)

In [ ]:
def similarity_from_reprojection(live_rgb, demo_rgb, demo_mask, return_images=False):
    # evaluate the similarity via flow reprojection error
    flow = servo_module.flow_module.step(demo_rgb, live_rgb)
    warped = servo_module.flow_module.warp_image(live_rgb / 255.0, flow)
    diff = (warped - (demo_rgb / 255.0))
    error = np.linalg.norm((warped - (demo_rgb / 255.0)), axis=2) * demo_mask
    error = error.sum() / demo_mask.sum()
    mean_flow = np.linalg.norm(flow[demo_mask],axis=1).mean()
    if return_images:
        return error, mean_flow, flow, warped
    return error, mean_flow

In [ ]:
from sklearn.preprocessing import minmax_scale

def normalize_errors(errors, flows):
    errors_l = errors[demo_good]
    mean_flows_l = flows[demo_good]
    errors_norm = np.ones(errors.shape)
    w = 0.5
    errors_norm[demo_good] = np.mean((1*minmax_scale(errors_l), w*minmax_scale(mean_flows_l)),axis=0)/(1+w)
    return errors_norm

## Error matrix between demonstration parts 

In [ ]:
import seaborn as sns

error_matrix = np.ones((len(playbacks), len(playbacks)))
flow_matrix = np.zeros((len(playbacks), len(playbacks)))

for k1, v1 in tqdm(demo_parts.items()):
    demo_i1 = int(k1)
    im1 = playbacks[demo_i1][v1[0]['end']].cam.get_image()[0]
    for k2, v2 in demo_parts.items():
        demo_i2 = int(k2)      
        im2 = playbacks[demo_i2][v2[1]['start']].cam.get_image()[0]
        mask2 = playbacks[demo_i2].fg_masks[v2[1]['start']]
        
        error, flow = similarity_from_reprojection(im1, im2, mask2)
        error_matrix[demo_i1, demo_i2] = error
        flow_matrix[demo_i1, demo_i2] = flow

error_matrix_norm = normalize_errors(error_matrix, flow_matrix)
np.savez(f"{root_dir}/error_matrix.npz", error_matrix_norm)

sns.heatmap(1 - error_matrix_norm)

## Errors with respect to goal image
We use a hacky function to update the segmentation mask of the goal image.
Additionally, we store errors for goal frames of all availbale shapes in their respective npz files

In [ ]:
from robot_io.recorder.simple_recorder import unprocess_seg

errors_rear = np.ones((len(playbacks), 1))
flows_rear = np.zeros((len(playbacks), 1))

for goal_rec in goal_recordings:
    if 'trapeze' in goal_rec:
        obj = 'trapeze'
        object_name = "Trapezium_Sort_Box"
    elif 'oval' in goal_rec:
        obj = 'oval'
        object_name = "Oval_Sort_Box"
        
    goal_pl = PlaybackEnvServo(goal_rec)

    info = goal_pl[-1].get_info()
    print(info.keys())
    name2uid = {}
    for i in range(10):
        i = str(i)
        if i not in info.keys():
            continue
        name2uid[info[i]["name"]] = info[i]["UID"]
    print(name2uid)
    obj_uid = int(name2uid[object_name])
    srf_uid = int(name2uid["surface_red"])
    seg_mask = info["seg_mask"]
    seg_obj, _ = unprocess_seg(seg_mask)
    
    # Get Goal Mask - insertion surface and object
    goal_mask = np.logical_or(seg_obj == obj_uid, seg_obj == srf_uid)
    
    # Get Goal RGB image
    goal_rgb = goal_pl[-1].cam.get_image()[0]

    for k1, v1 in demo_parts.items():
        demo_i1 = int(k1)
        im1 = playbacks[demo_i1][v1[1]['end']].cam.get_image()[0]
        error, flow = similarity_from_reprojection(im1, goal_rgb, goal_mask)
        errors_rear[demo_i1] = error
        flows_rear[demo_i1] = flow

    errors_rear_norm = normalize_errors(errors_rear, flows_rear)
    np.savez(f"{root_dir}/errors_rear_{obj}.npz", errors_rear_norm)

In [ ]:
# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0][-1].cam.get_image()[0])

def update(demo_index):
    image = playbacks[demo_index][-1].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print(f"Error: {errors_rear[demo_index]}, Error Norm: {errors_rear_norm[demo_index]}")
    print(playbacks[demo_index][-1].data['rew'])
    print(f"Good Demo: {demo_good[demo_index]}")
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)